In [1]:
# You may need to remove the '#' from the following commands to install the required dependencies

#these two are to read the excel
#! pip install xlrd
#! pip install install openpyxl


In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import sys

import statsmodels.formula.api as smf
from itertools import combinations

In [3]:
min_repetitions=2

In [4]:
# Defining the number of splits in the data
try:
    split_nr = int(sys.argv[1])
    n_splits = int(sys.argv[2])
except:
    split_nr = 1
    n_splits = 10
    print('Did you forget to add the split number and the number of splits?')
    #exit()


Did you forget to add the split number and the number of splits?


In [8]:
#One row per cell line
#DIR
x = pd.read_csv('data/CCLE_expression_proteincoding_genes_expected_count.csv', sep=',')#.fillna(0).drop(columns=['Project_Identifier'])
#c = [a.replace(';','.') for a in x.columns]
#x.columns = c
x.columns


Index(['Unnamed: 0', 'TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)',
       'SCYL3 (57147)', 'C1orf112 (55732)', 'FGR (2268)', 'CFH (3075)',
       'FUCA2 (2519)', 'GCLC (2729)',
       ...
       'H3C2 (8358)', 'H3C3 (8352)', 'AC098582.1 (8916)',
       'DUS4L-BCAP29 (115253422)', 'C8orf44-SGK3 (100533105)',
       'ELOA3B (728929)', 'NPBWR1 (2831)', 'ELOA3D (100506888)',
       'ELOA3 (162699)', 'CDR1 (1038)'],
      dtype='object', length=19222)

In [10]:
x.head()

,Unnamed: 0,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),...,H3C2 (8358),H3C3 (8352),AC098582.1 (8916),DUS4L-BCAP29 (115253422),C8orf44-SGK3 (100533105),ELOA3B (728929),NPBWR1 (2831),ELOA3D (100506888),ELOA3 (162699),CDR1 (1038)
0,ACH-001113,2383.0,0.0,5332.8,961.53,1518.50,2.0,160.96,566.38,9078.5,...,45.0,1.0,29.15,842.80,6.05,0.0,35.0,0.0,0.0,0.00
1,ACH-001289,2529.0,13.0,3978.0,555.83,832.17,0.0,19.16,999.84,1758.0,...,12.0,8.0,42.87,421.52,6.91,0.0,2.0,0.0,0.0,0.00
2,ACH-001339,1552.0,0.0,8303.0,849.46,2511.50,2.0,199.85,13850.00,2189.0,...,7.0,14.0,240.90,2460.00,11.97,0.0,15.0,0.0,0.0,1.06
3,ACH-001538,5657.0,0.0,6309.2,1056.50,656.47,0.0,7692.00,8454.40,3473.0,...,2.0,6.0,63.20,750.29,11.65,0.0,4.0,0.0,0.0,0.00
4,ACH-000242,22806.0,0.0,5090.6,1109.10,1499.90,98.0,31257.00,6449.60,22386.0,...,20.0,64.0,20.13,1101.30,0.00,0.0,100.0,0.0,0.0,0.00


In [15]:
#DIR
y = pd.read_csv('data/DrugResponse_PANCANCER_GDSC1_GDSC2_20200602.csv')[['drug_id','cell_line_name','ln_IC50']]
y.columns

/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['drug_id', 'cell_line_name', 'ln_IC50'], dtype='object')

In [14]:
a = pd.read_csv('data/sample_info.csv', sep=',')
#a.head()

a[a.cell_line_name.fillna("").apply(lambda x: 'TMK' in x)]

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
1692,ACH-002202,TMK-1,TMK1,TMK1_STOMACH,NaN,1299069.0,Male,Sanger,CVCL_4384,547.0,...,NaN,NaN,NaN,NaN,NaN,PT-xnqeDr,NaN,Gastric adenocarcinoma,C4004,NaN


In [23]:
(a.cell_line_name.unique())

1840

In [25]:
a[a.cell_line_name.duplicated(keep=False)].sort_values('cell_line_name')

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
147,ACH-001172,U-251 MG,U251MGDM,U251MGDM_Brain,NaN,NaN,Male,HSRRB,NaN,NaN,...,astrocytoma,NaN,NaN,NaN,NaN,PT-88IC1n,NaN,NaN,NaN,NaN
591,ACH-000232,U-251 MG,U251MG,U251MG_CENTRAL_NERVOUS_SYSTEM,U251-MG,905983.0,Male,HSRRB,CVCL_0021,1913.0,...,astrocytoma,NaN,NaN,NaN,NaN,PT-88IC1n,NaN,Astrocytoma,C60781,NaN
8,ACH-000071,NaN,HS706T,HS706T_BONE,NaN,NaN,Female,ATCC,CVCL_0863,NaN,...,NaN,NaN,2D: adherent,NaN,NaN,PT-HqO4KC,NaN,Malignancy in giant cell tumor of bone,C4304,NaN
12,ACH-000084,NaN,MUTZ3,MUTZ3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,MUTZ3,NaN,Male,DSMZ,CVCL_1433,NaN,...,NaN,NaN,2D: suspension,NaN,NaN,PT-S6ty7D,NaN,Acute myeloid leukemia with inv(3) (q21.3q26.2...,C82426,NaN
23,ACH-000165,NaN,HS751T,HS751T_FIBROBLAST,NaN,NaN,Male,ATCC,CVCL_0891,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-EyHts5,NaN,Hodgkin lymphoma,C9357,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,ACH-001214,NaN,U138MG,U138MG_CENTRAL_NERVOUS_SYSTEM,NaN,NaN,Male,ATCC,CVCL_0020,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-5XnRfQ,NaN,Astrocytoma,C60781,NaN
1376,ACH-001228,NaN,UPCISCC152,UPCISCC152_UPPER_AERODIGESTIVE_TRACT,"UPCISCC152, UPCI:SCC152",NaN,Male,Academic lab,CVCL_C058,NaN,...,oral,NaN,NaN,NaN,NaN,PT-k3C7C0,NaN,Tongue squamous cell carcinoma,C4648,NaN
1377,ACH-001232,NaN,UW228,UW228_CENTRAL_NERVOUS_SYSTEM,NaN,NaN,Male,Academic lab,CVCL_8585,NaN,...,NaN,non_MYC,NaN,NaN,NaN,PT-Qe9Gjx,NaN,Medulloblastoma,C3222,NaN
1380,ACH-001249,NaN,YMB1,YMB1_BREAST,NaN,NaN,Female,HSRRB,CVCL_2814,NaN,...,ERpos_HER2neg,luminal,NaN,NaN,NaN,PT-k1TO7o,ACH-000097,Invasive breast carcinoma of no special type,C4194,Contaminated. Shown to be a ZR-75-1 derivative...


In [16]:
y

,drug_id,cell_line_name,ln_IC50
0,1409,697,-5.452393
1,1057,697,-3.891777
2,1060,697,-0.170167
3,252,697,-2.762333
4,282,697,-2.615651
...,...,...,...
605251,2290,ZR-75-30,6.547607
605252,1735,ZR-75-30,3.195281
605253,1034,ZR-75-30,1.052725
605254,1862,ZR-75-30,1.570232


In [7]:
# This cell is the function to go from the table to a JSON file (variantSpark format and structure)

def merge_tree_node(tree, node):
    
    # Empty tree
    if type(tree)==float: return tree
    if len(tree)==0: return node

    # Direct children
    if tree['right'] == node['nodeID']:
        tree['right'] = node
        return tree
    elif tree['left'] == node['nodeID']:
        tree['left'] = node
        return tree

    # Create
    right = merge_tree_node(tree['right'], node)
    left = merge_tree_node(tree['left'], node)
    tree['right'] = right
    tree['left'] = left
    return tree
            

def from_table_to_json(m):
    tree = {}
    for _id,row in m.iterrows():
        current_node = {'nodeID': row['nodeID'], 
                        'splitvarID':row['splitvarID'],
                        'splitVar':row['splitvarName'],
                        'splitval':row['splitval'], 
                        'terminal':row['terminal'], 
                        'prediction':row['prediction'], 
                        'left':row['leftChild'], 
                        'right':row['rightChild'] }
        tree = merge_tree_node(tree, current_node)
    return tree



# Test
#m = pd.read_csv('output/tree1.csv')
#from_table_to_json(m)

In [8]:
# Algorithm to get the interacting nodes (no testing done yet)

def get_interactions(tree, current_list, interactions):
    if not 'splitVar' in tree.keys():
        return 0
    if str(tree['splitVar']) == 'nan': return 0 #ranger adds a fake predicting node at the end
    
    # Adding the interaction
    current_list.append(tree['splitVar'])
    if len(current_list) >= 2:
        for i in range(2,len(current_list)+1):
            aux = '+'.join(sorted(current_list[-i:]))
            if aux in interactions.keys():
                interactions[aux] +=1
            else:
                interactions[aux] = 1
                    
    if 'left' in tree.keys():
        get_interactions(tree['left'], current_list, interactions)
    if 'right' in tree.keys():
        get_interactions(tree['right'], current_list, interactions)
        
    _ = current_list.pop()
    

In [9]:
# Testing all the interactions

def test_interactions(df, data):
    """
    I use GLM because:
    The main difference between the two approaches is that the general linear model strictly assumes that
    the residuals will follow a conditionally normal distribution, while the GLM loosens this assumption 
    and allows for a variety of other distributions from the exponential family for the residuals.
    """
    final_results = []
    counts = 0

    for v in df[(df.repetitions>=2) & (df.order ==2)].variants.tolist():
        
        #preparing the input
        sp=v.split('+')
        if len(sp) != len(set(sp)): continue
        xy = data[sp+['ln_IC50']].fillna(-1)
        sp=v.replace('_','').split('+')
        xy.columns = [''.join([chr(int(y)+97) if y.isnumeric() else y for y in x.replace('_','').replace('.','')]) for x in xy.columns]
        formula = xy.columns[-1]+' ~ '
        for i in range(1,len(xy.columns)):
            formula = formula + ' + '.join(['*'.join(o) for o in list(combinations(xy.columns[:-1],i))])
            formula = formula + ' + '
        formula = formula.rstrip(' + ')

        # Standard fitting
        ols = smf.ols(formula,data=xy)
        ols.raise_on_perfect_prediction = False #preventing the perfect separation error
        results = ols.fit(disp=False, maxiter=1000) #mehtod prevents singular matrix
        results = results.summary()
        converged = results.tables[0].data[5][1].strip()
        pseudo_r2 = results.tables[0].data[3][3].strip()
        results = results.tables[1].data
        results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
        results['standard_fitting'] = True

        #If nan means no convergence bc singular matrix
        #adding regularization
        if 'nan' == pd.DataFrame(results)['z'].iloc[2].strip():
            try:
                results = ols.fit_regularized(method='l1', disp=False, maxiter=1000, alpha=0.3) #mehtod prevents singular matrix
                results = results.summary()
                converged = results.tables[0].data[5][1].strip()
                pseudo_r2 = results.tables[0].data[3][3].strip()
                results = results.tables[1].data
                results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
                results['standard_fitting'] = False        
            except:
                #crashed the regularized
                counts +=1

        results['converged'] = converged
        results['pseudo_r2'] = pseudo_r2
        results['snps'] = v
        results['order'] = len(sp)
        final_results.append(results)

    final_results = pd.concat(final_results)
    #print(counts)
    return final_results

In [10]:
# Testing all the interactions

def results_fit_to_df(results):
    coeffs = results.params.tolist()
    pvals = results.pvalues.tolist()
    pseudo_r2 = results.rsquared
    tvals = results.tvalues.tolist()
    cint_low = results.conf_int()[0].tolist()
    cint_high = results.conf_int()[1].tolist()

    results = results.summary()
    converged = results.tables[0].data[5][1].strip()
    results = results.tables[1].data
    results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
    results['P>|z|'] = pvals
    results['z'] = tvals 
    results['coef'] = coeffs
    results['converged'] = converged
    results['pseudo_r2'] = pseudo_r2
    results['[0.025'] = cint_low
    results['0.975]'] = cint_high
    return results
    
def test_interactions_high(df, data, max_order=4):
    """
    I use GLM because:
    The main difference between the two approaches is that the general linear model strictly assumes that
    the residuals will follow a conditionally normal distribution, while the GLM loosens this assumption 
    and allows for a variety of other distributions from the exponential family for the residuals.
    """
    final_results = []
    counts = 0

    for m_or in range(2,max_order+1):
        #print('current order',m_or)
        
        for v in df[(df.repetitions>=2) & (df.order==m_or)].variants.tolist():
            #preparing the input
            sp=v.split('+')
            xy = data[sp+['ln_IC50']].fillna(-1)
            sp=v.replace('_','').split('+')
            xy.columns = [''.join([chr(int(y)+97) if y.isnumeric() else y for y in x.replace('_','').replace('.','')]) for x in xy.columns]
            formula = xy.columns[-1]+' ~ '
            for i in range(1,len(xy.columns)):
                formula = formula + ' + '.join(['*'.join(o) for o in list(combinations(xy.columns[:-1],i))])
                formula = formula + ' + '
            formula = formula.rstrip(' + ')
            
            #Recreating the formula
            if m_or>2:
                #gathering all interactions
                fs = formula.split(' + ')
                formula = ' + '.join([a for a in fs if '*' not in a]+[a for a in fs if a.count('*')== m_or-1])
                all_interactions = [a.replace('*',':') for a in fs if '*' in a]
                final_results = pd.concat(final_results)
                subset = final_results[final_results.coef_id.apply(lambda a: a in all_interactions)].reset_index(drop=True)
                final_results = [final_results]
                if len(subset)>0:
                    max_idx = subset['coef'].astype(float).abs().idxmax()
                    coef_id = subset.loc[max_idx].coef_id
                    formula = formula +' + '+coef_id.replace(':','*')
                else:
                    #pass
                    continue # bc i dont think it is a valid tree form (interaction-wise)
                    #There is no sub epistasis (P>Q>O>P, tree 503, first compound)

            # Standard fitting
            try:
                ols = smf.ols(formula.replace('*',':'),data=xy)
                # "*" vs ":" #https://stackoverflow.com/questions/33050104/difference-between-the-interaction-and-term-for-formulas-in-statsmodels-ols
            except:
                print('error in OLS')
                print('coef_id',coef_id)
                print('formula OLS',type(formula),formula)
                #return pd.concat(final_results)
                continue
            ols.raise_on_perfect_prediction = False #preventing the perfect separation error
            results = ols.fit(disp=False, maxiter=1000) #mehtod prevents singular matrix
#            return results
            results = results_fit_to_df(results)
            results['standard_fitting'] = True

            #If nan means no convergence bc singular matrix
            #adding regularization
            if 'nan' == pd.DataFrame(results)['z'].astype(str).iloc[2].strip():
                try:
                    results = ols.fit_regularized(method='l1', disp=False, maxiter=1000, alpha=0.3) #mehtod prevents singular matrix
                    results = results_fit_to_df(results)
                    results['standard_fitting'] = False        
                except:
                    #crashed the regularized
                    counts +=1
                    continue


            results['snps'] = v
            results['order'] = len(sp)
            final_results.append(results)

    final_results = pd.concat(final_results)
    return final_results


In [11]:
def undo(string):
    
    string = ''.join([ x if ord(x)<90 else str(ord(x)-97) for x in string ])
    string = string[:6]+'.'+string[6:].replace('HUMAN', '_HUMAN') #not sure these 6
    return string
    
#undo('PacfbbCRYABHUMAN')

In [12]:
#TODO: By compound

#Looping over all drugs
# drug_id
# Other options:
# - drug_name
# - CHEMBL = Chemical compound ID
#for compound_name, group in x.merge(y, left_on='Cell_Line', right_on='cell_line_name').groupby('drug_id'): # may require too much memory

#Making a temp file to run all R stuff
#DIR
os.system(f"mkdir -p tmp/tmp{split_nr}")


column_to_group = 'drug_id'
drugs_list = y[column_to_group].sort_values().unique()
drugs_list = [drugs_list[i] for i in range(len(drugs_list)) if i%n_splits==split_nr]
i = -1
all_drug_results = []
for elm in drugs_list[:3]:
    i+=1
    
    if i%10==0 or i<10: print(i,'out of',len(drugs_list), 'drugs in split nr', split_nr)

    xy = x.merge(y[y[column_to_group]==elm], left_on='Cell_Line', right_on='cell_line_name')
    #Enhancement: Remove peptides that are all zero 
    
    # saving csv for R df
    # file name is generic but we could personalize it
    #DIR
    xy.drop(columns=['Cell_Line', 'cell_line_name','drug_id']).rename(columns={'ln_IC50':'label'}).to_csv(f"tmp/tmp{split_nr}/data.csv", index=False)

    #Run the R script to generate the outputs
    os.system(f"Rscript ranger_run.R {split_nr}")
    
    #load the R outputs (the trees, one file each), and convert it to VS look-alike and get interactions
    interactions = {}
    #DIR
    trees = os.listdir(f"tmp/tmp{split_nr}/")
    #files = [x for x in files if 'tree' in x]
    for tree in trees:
        if 'tree' not in tree: continue #if it is not a tree file ignore
        #DIR
        tree_json = from_table_to_json(pd.read_csv(f"tmp/tmp{split_nr}/"+tree))        
        get_interactions(tree_json,[],interactions) #the interactions are found in "interactions"
        
    # Creating a df out of the interactions
    df = pd.DataFrame({'variants':interactions.keys(),'repetitions':interactions.values()})
    df['order'] = df.variants.apply(lambda x: x.count('+')+1)
    
    
    tested_interactions = test_interactions_high(df, xy) #here you define which order of interactions you want to compute
    tested_interactions['drug'] = elm
    all_drug_results.append(tested_interactions)
    

    

final_results = pd.concat(all_drug_results)
#DIR
final_results.to_csv(f"tmp/final_results{split_nr}.tsv", index=False, sep='\t')

0 out of 75
current order 2
current order 3
current order 4
1 out of 75
current order 2
current order 3
current order 4
2 out of 75
current order 2
current order 3
current order 4


In [13]:
print('Done:',split_nr)

Done: 1


In [16]:
x.shape

(949, 6693)